In [1]:
import pandas as pd
from os import listdir
import numpy as np

In [2]:
files = listdir('dataset/ExtraSensory/ExtraSensory.per_uuid_features_labels/')

In [3]:
keep_files = []
labels_lying_down = []

for file in files:
    user = pd.read_csv('dataset/ExtraSensory/ExtraSensory.per_uuid_features_labels/' + file)
    prop_na = user.isnull().sum().sum() / (user.shape[0] * user.shape[1])
    if prop_na < 0.3:
        keep_files.append(file)
        labels_lying_down.append(user['label:LYING_DOWN'])

labels_lying_down = pd.concat(labels_lying_down, axis=0, ignore_index=True)

magnet_columns= [col for col in user.columns if 'raw_magnet:' in col]
watch_columns= [col for col in user.columns if 'watch_acceleration:' in col]
location_quick_columns = [col for col in user.columns if 'location_quick_features:' in col]
del user, prop_na

In [4]:
# label lying down interpoleren
# Op basis van tijd en label watch values op 0 zetten

dir = 'dataset/ExtraSensory/ExtraSensory.per_uuid_features_labels/'

total = []

for file in keep_files:
    name = file.split('.')[0]

    t = pd.read_csv(dir + file)
    t['user'] = name

    t['label:LYING_DOWN'].interpolate(method='nearest', inplace=True)

    for watch_col in watch_columns:

        t[watch_col] = t.apply(
        lambda row: 0 if (np.isnan(row[watch_col]) and row['label:LYING_DOWN'] ==1.0 and
                          (row[watch_col] ==1.0 or row[watch_col])
                          ) else row[watch_col],
        axis=1)
        t[watch_col].interpolate(method='linear', inplace=True)
        t[watch_col].interpolate(method='nearest', inplace=True) # Some start with nan values


    for magnet_col in magnet_columns:
        t[magnet_col].interpolate(method='linear', inplace= True)
        t[magnet_col].interpolate(method='nearest', inplace= True)

    for location_col in location_quick_columns:
        t[location_col].interpolate(method='linear', inplace= True)
        t[location_col].interpolate(method='nearest', inplace= True)

    t['audio_properties:max_abs_value'].interpolate(method='linear', inplace= True)
    t['audio_properties:max_abs_value'].interpolate(method='nearest', inplace=True)

    total.append(t)

total = pd.concat(total, axis=0, ignore_index=True)
total['label:LYING_DOWN'] = labels_lying_down

In [5]:
drop_cols = (total.isnull().sum() / total.shape[0]) > 0.5
drop_cols = drop_cols[drop_cols == True].index
drop_cols = [col for col in drop_cols if 'label:' not in col]
total = total.drop(columns=drop_cols)

# We drop location since we have location quick
location_columns= [col for col in total.columns if 'location:' in col]
total = total.drop(columns= location_columns)

audio_columns= [col for col in total.columns if 'audio_naive:' in col]
total= total.drop(columns= audio_columns)

total = total.drop(columns=['lf_measurements:proximity', 'lf_measurements:screen_brightness', 'audio_properties:normalization_multiplier'])

In [8]:
for col in total.columns:
    if col != 'user':
        total[col].fillna(value=total[col].median(), inplace=True)

In [23]:
# Now group by name and save seperate dataframe
for i, df_user in total.groupby('user'):
    name = df_user['user'].iloc[0]
    df_user = df_user.drop(columns=['user'])
    df_user.to_csv('dataset/ExtraSensory/Processed/' + name + '.csv', index=False)

In [22]:
df_user['user'].iloc[0]

'0A986513-7828-4D53-AA1F-E02D6DF9561B'

In [24]:
total.isna().sum()

timestamp                          0
raw_acc:magnitude_stats:mean       0
raw_acc:magnitude_stats:std        0
raw_acc:magnitude_stats:moment3    0
raw_acc:magnitude_stats:moment4    0
                                  ..
label:PHONE_ON_TABLE               0
label:WITH_CO-WORKERS              0
label:WITH_FRIENDS                 0
label_source                       0
user                               0
Length: 225, dtype: int64